# 데이터 로드

In [1]:
import os
import json
import pandas as pd # Gzip 압축된 CSV 파일 불러오기

df = pd.read_csv("data.csv.gz", compression='gzip')
df.head()

,standard_form,dialect_form
0,난 진짜 벌초 할때 뱀나오잖아 뱀도,난 진짜 벌초 할때 뱀나오잖아 뱀도
1,어어 그니까,어어 그니까
2,아 잘도,아 잘도
3,예초기 뱀도 짤려,예초기 뱀도 짤려
4,어 그니까 완전 끔찍해,어 그니까 완전 끔찍해


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2774257 entries, 0 to 2774256
Data columns (total 2 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   standard_form  object
 1   dialect_form   object
dtypes: object(2)
memory usage: 42.3+ MB


In [3]:
# 중복 제거
df_filtered = df[df["standard_form"] != df["dialect_form"]]
df_filtered = df_filtered.drop_duplicates()
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1116389 entries, 6 to 2774254
Data columns (total 2 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   standard_form  1116389 non-null  object
 1   dialect_form   1116389 non-null  object
dtypes: object(2)
memory usage: 25.6+ MB


In [4]:
df_filtered.reset_index(drop=True, inplace=True)

import re
SPECIALS = "".join([".", ",", ";", ":", "!", "?", '"', "'", "&", "#", "$", "%"])

def preprocess(text: str, only_kor: bool = True):
    """한국어 문장을 옵션에 맞게 전처리"""
    # 한국어 모음과 특수문자, 숫자 및 영어 제거
    if only_kor:
        text = re.sub(r"[^가-힣\s]+", "", text)
    else:
        text = re.sub(rf"[^가-힣ㄱ-ㅎ0-9{SPECIALS}a-zA-Z\s]+", "", text)

    # 연속 공백 제거
    text = re.sub(r"\s+", " ", text)

    # 좌우 불필요한 공백 제거
    return text.strip()


df_filtered['dialect_form'] = df_filtered['dialect_form'].apply(preprocess)
df_filtered['standard_form'] = df_filtered['standard_form'].apply(preprocess)

df_filtered.head()

,standard_form,dialect_form
0,예전에 그렇게,예전에 겅
1,아 그러니까 예초 할땐 조심해야 돼,아 겅하니까이 예초 할땐 조심해야 돼
2,그러고 예초 벌초하다그네하다가,겅허고 예초 벌초하다그네하다가
3,어 그거 튀겨서,어 그거 튀겨그네
4,돌멩이 해 가지고 실명된사람 있어,돌멩이 해 가지고 실명된사람 이서


## 데이터 split

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터프레임 섞기
df_shuffled = df_filtered.sample(frac=1, random_state=42).reset_index(drop=True)

# 80%를 훈련 세트로, 20%를 테스트 세트로
train_df, test_df = train_test_split(df_shuffled, test_size=0.2, random_state=42)

# 결과 확인
print("훈련 세트 크기:", train_df.shape)
print("테스트 세트 크기:", test_df.shape)

훈련 세트 크기: (893111, 2)
테스트 세트 크기: (223278, 2)


In [6]:
train_df.head()

,standard_form,dialect_form
439450,그러고 디스플레이 부족하댄허난하다고 하니까 사람을 구해준 것 아닙니까,겅허고 디스플레이 부족하댄허난하다고 하니까 사람을 구해준 것 아니꽈
981560,너 종합병원이야 갑자기 다리꼬지마,너 종합병원 갑자기 다리꼬지마
1084245,그러니까 요즘에 오는 관광객들,게난 요즘에 오는 관광객들
97854,그렇게 하니까 한번 해볼까 해서 그냥 호기심으로 했지,경 하니까 한번 해보카 해연에 그냥 호기심으로 했주게
975837,오들락오들락 잘 나 정말 곱게 난다 잡초 하나 없이,오들락오들락 잘 나메 정말 곱게 난다 검질 호나 어시


In [7]:
test_df.head()

,standard_form,dialect_form
552182,근데 그거 할려면은 몸 좋아야 되지 않아,겐디 그거 할려면은 몸 좋아야 되지 않
920955,혼자 늙으면 이거 안될 수도 있을 건가,혼자 늙으면 이거 안될 수도 이실 건가
621006,그렇게할 거 같아 어떻게 할거야,겅할 거 닮아 어떵할거
778371,쫌 쫌 이렇게 삶을 평탄하게 사는 것 같대 물 흐르듯 물 흐르듯이 자연스럽게 사는 ...,쫌 쫌 이렇게 삶을 평탄하게 사는 것 같댄 물 흐르듯 물 흐르듯이 자연스럽게 사는 ...
579293,고민하고있어 어떡할지,고민하맨 어떵하코


## 데이터 별 유사도 계산

In [8]:
import pandas as pd
import spacy

# spaCy 한국어 모델 로드
nlp = spacy.load('ko_core_news_sm')

def calculate_similarity(row):
    # 표준어와 방언 문장을 spaCy 문서 객체로 변환
    doc1 = nlp(row['standard_form'])
    doc2 = nlp(row['dialect_form'])
    
    # 두 문장 간의 유사도 계산
    similarity = doc1.similarity(doc2)
    
    return similarity

In [9]:
# 유사도 계산 및 새로운 열 추가
train_df['similarity'] = train_df.apply(calculate_similarity, axis=1)

# 유사도를 기준으로 내림차순 정렬
# df_sorted = train_df.sort_values(by='similarity', ascending=False)

# 결과 출력
print(train_df[['standard_form','dialect_form', 'similarity']])

# 결과를 CSV 파일로 저장
train_df.to_csv('jeju_train_similarity.csv', encoding='utf-8-sig', index=False)

C:\Users\user\AppData\Local\Temp\ipykernel_165304\4259840709.py:13: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = doc1.similarity(doc2)
C:\Users\user\AppData\Local\Temp\ipykernel_165304\4259840709.py:13: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = doc1.similarity(doc2)


                                  standard_form  \
439450   그러고 디스플레이 부족하댄허난하다고 하니까 사람을 구해준 것 아닙니까   
981560                       너 종합병원이야 갑자기 다리꼬지마   
1084245                        그러니까 요즘에 오는 관광객들   
97854             그렇게 하니까 한번 해볼까 해서 그냥 호기심으로 했지   
975837             오들락오들락 잘 나 정말 곱게 난다 잡초 하나 없이   
...                                         ...   
110268                군대 가면 옥택연 어 옥택연을 부담스럽지 않아   
259178           중저가 브랜드여도 이름 있는 데 거 해가지고 하면은 이   
131932              거기까진 아닌가봐 아직까지는 교육도 시키고 해야지   
671155                       그거 얘기는 웬만하면 말하지 마라   
121958                                 정체가 뭔데 너   

                                  dialect_form  similarity  
439450   겅허고 디스플레이 부족하댄허난하다고 하니까 사람을 구해준 것 아니꽈    0.855086  
981560                        너 종합병원 갑자기 다리꼬지마    0.835181  
1084245                         게난 요즘에 오는 관광객들    0.734492  
97854            경 하니까 한번 해보카 해연에 그냥 호기심으로 했주게    0.725456  
975837           오들락오들락 잘 나메 정말 곱게 난다 검질 호나 어시    0.880031  
...                  

In [10]:
# 유사도 계산 및 새로운 열 추가
test_df['similarity'] = test_df.apply(calculate_similarity, axis=1)

# 유사도를 기준으로 내림차순 정렬
# df_sorted = train_df.sort_values(by='similarity', ascending=False)

# 결과 출력
print(test_df[['standard_form','dialect_form', 'similarity']])

# 결과를 CSV 파일로 저장
test_df.to_csv('jeju_test_similarity.csv', encoding='utf-8-sig', index=False)

C:\Users\user\AppData\Local\Temp\ipykernel_165304\4259840709.py:13: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = doc1.similarity(doc2)


                                            standard_form  \
552182                             근데 그거 할려면은 몸 좋아야 되지 않아   
920955                              혼자 늙으면 이거 안될 수도 있을 건가   
621006                                  그렇게할 거 같아 어떻게 할거야   
778371  쫌 쫌 이렇게 삶을 평탄하게 사는 것 같대 물 흐르듯 물 흐르듯이 자연스럽게 사는 ...   
579293                                        고민하고있어 어떡할지   
...                                                   ...   
527610        내가 말했지 딱 나 퇴근하기 십분 전까지 아무것도 못 판 거야 그래서 언니한테   
488777                                   진짜 너 고생하네 고생했어 너   
215914                                               까먹었어   
873200                                       혹시 너 연개소문 봤어   
440515                                  일요일 월요일날 아침에 터졌었어   

                                             dialect_form  similarity  
552182                              겐디 그거 할려면은 몸 좋아야 되지 않    0.928874  
920955                              혼자 늙으면 이거 안될 수도 이실 건가    0.910187  
621006                                       겅할 거 닮

In [11]:
train_average_similarity = train_df['similarity'].mean()
test_average_similarity = test_df['similarity'].mean()

print(f"train_data의 'similarity' 열의 평균값: {train_average_similarity}")
print(f"test_data의 'similarity' 열의 평균값: {test_average_similarity}")

train_data의 'similarity' 열의 평균값: 0.8059610633317846
test_data의 'similarity' 열의 평균값: 0.8064080678530636
